In [ ]:
import numpy as np
import tensorflow as tf
import pandas as pd
import matplotlib.pyplot as plt
import os
import cv2
from google.colab import drive
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

# Mount Google Drive
drive.mount('/content/drive')

dataset_path = '/content/drive/My Drive/dataset'

# Read dataset
class_labels = []
for item in os.listdir(dataset_path):
    item_path = os.path.join(dataset_path, item)

    if os.path.isdir(item_path):
        image_count = 0
        for image_name in os.listdir(item_path):
            image_path = os.path.join(item_path, image_name)

            if os.path.isfile(image_path):
                if item == "Non-Frog" and image_count >= 253:
                    continue
                class_labels.append((item, image_path))  # Store full path
                if item == "Non-Frog":
                    image_count += 1

# Convert to DataFrame
df = pd.DataFrame(class_labels, columns=["Labels", "Image"])
print(df.head(), df.tail(), df['Labels'].value_counts())

# Encode labels
label_encoder = LabelEncoder()
df['Labels'] = label_encoder.fit_transform(df['Labels'])

# Load & resize images
im_size = 224
images = []
labels = []

for _, row in df.iterrows():
    img = cv2.imread(row['Image'])
    if img is not None:
        img = cv2.resize(img, (im_size, im_size))
        images.append(img)
        labels.append(row['Labels'])

images = np.array(images)
labels = np.array(labels).reshape(-1, 1)

# One-hot encode labels
onehot_encoder = OneHotEncoder(sparse_output=False)
Y = onehot_encoder.fit_transform(labels)

# Shuffle & split dataset
images, Y = shuffle(images, Y, random_state=1)
x_train, x_test, y_train, y_test = train_test_split(images, Y, test_size=0.10, random_state=42)

# Normalize after splitting
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0

print(x_train.shape, y_train.shape, x_test.shape, y_test.shape)


Mounted at /content/drive
     Labels                                              Image
0  Non-Frog  /content/drive/My Drive/dataset/Non-Frog/RCNX0...
1  Non-Frog  /content/drive/My Drive/dataset/Non-Frog/RCNX0...
2  Non-Frog  /content/drive/My Drive/dataset/Non-Frog/RCNX0...
3  Non-Frog  /content/drive/My Drive/dataset/Non-Frog/RCNX0...
4  Non-Frog  /content/drive/My Drive/dataset/Non-Frog/RCNX0...     Labels                                              Image
501   Frog  /content/drive/My Drive/dataset/Frog/RCNX0246.JPG
502   Frog  /content/drive/My Drive/dataset/Frog/RCNX0094.JPG
503   Frog  /content/drive/My Drive/dataset/Frog/RCNX0244.JPG
504   Frog  /content/drive/My Drive/dataset/Frog/RCNX0245.JPG
505   Frog  /content/drive/My Drive/dataset/Frog/RCNX2144.JPG Labels
Non-Frog    253
Frog        253
Name: count, dtype: int64
(455, 224, 224, 3) (455, 2) (51, 224, 224, 3) (51, 2)


In [ ]:
from tensorflow.keras import layers
from tensorflow.keras.applications import EfficientNetB0

NUM_CLASSES = 2
IMG_SIZE = 224
size = (IMG_SIZE, IMG_SIZE)

inputs = layers.Input(shape=(IMG_SIZE, IMG_SIZE, 3))
x = inputs

# EfficientNetB0
outputs = EfficientNetB0(include_top=True, weights=None, classes=NUM_CLASSES) (inputs)

In [ ]:
model = tf.keras.Model(inputs, outputs)

model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

hist = model.fit(x_train, y_train, epochs=20, batch_size=16, validation_data=(x_test, y_test))

Epoch 1/20
29/29 ━━━━━━━━━━━━━━━━━━━━ 134s 2s/step - accuracy: 0.8430 - loss: 1.6529 - val_accuracy: 0.5686 - val_loss: 3.3075
Epoch 2/20
29/29 ━━━━━━━━━━━━━━━━━━━━ 51s 74ms/step - accuracy: 0.9157 - loss: 1.2443 - val_accuracy: 0.5686 - val_loss: 4.6872
Epoch 3/20
29/29 ━━━━━━━━━━━━━━━━━━━━ 2s 70ms/step - accuracy: 0.9195 - loss: 0.5864 - val_accuracy: 0.5686 - val_loss: 3.8601
Epoch 4/20
29/29 ━━━━━━━━━━━━━━━━━━━━ 2s 70ms/step - accuracy: 0.9710 - loss: 0.0679 - val_accuracy: 0.5686 - val_loss: 4.3158
Epoch 5/20
29/29 ━━━━━━━━━━━━━━━━━━━━ 2s 74ms/step - accuracy: 0.9808 - loss: 0.0708 - val_accuracy: 0.5686 - val_loss: 4.2251
Epoch 6/20
29/29 ━━━━━━━━━━━━━━━━━━━━ 2s 75ms/step - accuracy: 0.9838 - loss: 0.0475 - val_accuracy: 0.5686 - val_loss: 4.4347
Epoch 7/20
29/29 ━━━━━━━━━━━━━━━━━━━━ 2s 72ms/step - accuracy: 0.9797 - loss: 0.0773 - val_accuracy: 0.5686 - val_loss: 2.6879
Epoch 8/20
29/29 ━━━━━━━━━━━━━━━━━━━━ 3s 72ms/step - accuracy: 0.9799 - loss: 0.0475 - val_accuracy: 0.5686 - 

In [ ]:
# Evaluate model
test_loss, test_acc = model.evaluate(x_test, y_test)
print("Test accuracy:", test_acc)

# Get predictions
y_pred = np.argmax(model.predict(x_test), axis=1)
y_true = np.argmax(y_test, axis=1)

# Compute separate accuracy for frog (label 1) and non-frog (label 0)
frog_correct = np.sum((y_pred == 1) & (y_true == 1))
frog_total = np.sum(y_true == 1)
non_frog_correct = np.sum((y_pred == 0) & (y_true == 0))
non_frog_total = np.sum(y_true == 0)

frog_accuracy = 100 * (frog_correct / frog_total) if frog_total > 0 else 0
non_frog_accuracy = 100 * (non_frog_correct / non_frog_total) if non_frog_total > 0 else 0

print(f"Frog Accuracy: {frog_accuracy:.2f}%")
print(f"Non-Frog Accuracy: {non_frog_accuracy:.2f}%")

2/2 ━━━━━━━━━━━━━━━━━━━━ 12s 5s/step - accuracy: 0.5666 - loss: 3.8219
Test accuracy: 0.5686274766921997
2/2 ━━━━━━━━━━━━━━━━━━━━ 13s 7s/step
Frog Accuracy: 0.00%
Non-Frog Accuracy: 100.00%
